In [4]:
from nltk.tokenize import PunktSentenceTokenizer, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
import re
from collections import Counter
from collections import defaultdict
import networkx as nx

In [11]:
def kw_extract(text):
    processed_text = [preprocess(s) for s in PunktSentenceTokenizer().tokenize(text)]
    sents = [sent.split() for sent in processed_text]

    idx_to_vocab, vocab_to_idx = scan_vocabulary(sents, min_count=2)
    graph = cooccurrence(sents, vocab_to_idx, window=8, min_cooccurrence=2)
    scores = calculate_score(graph)

    R = sorted(scores.items(), key=lambda x: -x[1])[:10]
    KeyWords = ', '.join([idx_to_vocab[idx] for idx, _ in R])

    return KeyWords

def preprocess(sentence):
    sentence = sentence.lower() 
    sentence = ''.join([c for c in sentence if c.isalnum() or c.isspace()])
    
    stop_words = set(stopwords.words('english'))
    words = word_tokenize(sentence)
    filtered_words = [word for word in words if word not in stop_words and word]
    
    pos_tags = pos_tag(filtered_words)
    pos_tags = [(word, 'a') if pos.startswith('J') else (word, pos.lower()) for word, pos in pos_tags]

    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word, pos=tag[0]) if tag[0] in ['n', 'v', 'a', 'r', ] else word for word, tag in pos_tags]
    
    return ' '.join(words)

def scan_vocabulary(sents, min_count=2):
    counter = Counter(w for sent in sents for w in sent)
    counter = {w:c for w,c in counter.items() if c >= min_count}
    idx_to_vocab = [w for w, _ in sorted(counter.items(), key=lambda x:-x[1])]
    vocab_to_idx = {vocab:idx for idx, vocab in enumerate(idx_to_vocab)}
    return idx_to_vocab, vocab_to_idx

def cooccurrence(tokens, vocab_to_idx, window=2, min_cooccurrence=2):
    counter = defaultdict(int)
    for s, tokens_i in enumerate(tokens):
        vocabs = [vocab_to_idx[w] for w in tokens_i if w in vocab_to_idx]
        n = len(vocabs)
        for i, v in enumerate(vocabs):
            if window <= 0:
                b, e = 0, n
            else:
                b = max(0, i - window)
                e = min(i + window, n)
            for j in range(b, e):
                if i == j:
                    continue
                counter[(v, vocabs[j])] += 1
                counter[(vocabs[j], v)] += 1
    counter = {k:v for k,v in counter.items() if v >= min_cooccurrence}
    
    nx_graph = nx.Graph()
    for (word1, word2), count in counter.items():
        nx_graph.add_edge(word1, word2, weight=count)
        
    return nx_graph

def calculate_score(graph):
    scores = nx.pagerank(graph)
    return scores

In [12]:
text = '''CABERNET SAUVIGNON
The world’s best-travelled red wine variety making concentrated, tannic wines for particularly long ageing.
Berry colour: black
PRINCIPAL SYNONYMS: Bidure (Graves), Bordeaux (Switzerland), Bordo (Romania), Bouchet or Bouchet Sauvignon or Petit Bouchet (Saint-Émilion and Pomerol), Burdeos Tinto (Spain), Cabernet Petit, Carbonet or Carbouet (Bazadais and Petites Graves), Carmenet, Lafit or Lafite (Bulgaria, Russia and the Republic of Moldova), Marchoupet (Castillon), Navarre (Dordogne), Petit Cabernet *, Petit Cavernet Sauvignon (Pauillac), Sauvignon (Médoc), Sauvignonne (Graves), Vidure or Vidure Sauvignonne or Petite Vidure (Graves)
VARIETIES COMMONLY MISTAKEN FOR CABERNET SAUVIGNON: CABERNET FRANC *, CARMENÈRE *
ORIGINS AND PARENTAGE
Cabernet Sauvignon comes from the Gironde in south-west France. The earliest mention of the variety appeared under the name Petit Cabernet in the ‘Livre de raison d’Antoine Feuilhade’, an account book written between 1763 and 1777 by the mayor of Libourne (Garde 1946). It was then listed in a catalogue of grape varieties established in Pauillac by Dupré de Saint-Maur in 1784: ‘Gros cavernet sauvignon, noir, première qualité pour faire du bon vin, assé productif & coloré, mais sujet au coulage. Petit cavernet sauvignon, noir, seconde qualité pour faire du bon vin, agreable & delicat, moins productif que le precedent’ (‘Gros cavernet sauvignon, black, top quality for making good wine, rather productive and deeply coloured, but subject to coulure. Petit cavernet sauvignon, black, secondary quality for making good wine, agreeable and delicate, less productive than the former’), and the first
     
mention of the modern Cabernet Sauvignon spelling did not appear until 1840 (Rézeau 1997).
Until the end of the nineteenth century, Cabernet Sauvignon was possibly confused with CABERNET FRANC under old synonyms such as Cabrunet, Carmenet, Vidure, or simply the French plural Cabernets (see Cabernet Franc). It is also possible that Cabernet Sauvignon simply did not exist
before the eighteenth century.
The parentage of Cabernet Sauvignon was fortuitously discovered by Carole Meredith and her
PhD student John Bowers at the University of California at Davis as recently as 1996. While generating a database for the DNA profiles of the most important grape varieties in their collection, Napa Valley-born Bowers noticed that the DNA profile of Cabernet Sauvignon was perfectly consistent with its being an offspring of CABERNET FRANC and SAUVIGNON BLANC (Bowers and Meredith 1997). This was the first time anyone had identified the parents of a classic wine grape variety, and it came as a big surprise to the wine world. Although ampelographers had already noticed many morphological similarities between both Cabernets, and although Cabernet Sauvignon apparently got its name because its wood and leaves resemble those of Sauvignon Blanc (Rézeau 1997), this parentage was totally unexpected, not least because no one had ever thought that a black- berried variety could have one white-berried parent. Gagnaire (1872) wrote at length in defence of this impossibility.
The cross that gave birth to Cabernet Sauvignon was most likely spontaneous and occurred somewhere in the Gironde before the mid eighteenth century. Since MERLOT is also a progeny of CABERNET FRANC, Cabernet Sauvignon and Merlot turn out to be half-siblings (see pedigree diagram overleaf). Most of these varieties belong to the Carmenet ampelographic group (see p XXVII). In addition, a probabilistic analysis of DNA data has shown that SAUVIGNON BLANC and CHENIN BLANC are siblings – the offspring of SAVAGNIN and an unknown parent (Vouillamoz). This makes Cabernet Sauvignon a nephew/niece of Chenin Blanc from the Loire and a grandchild of Savagnin from north-east France.
A pinky bronze-berried mutation of Cabernet Sauvignon was observed in 1977 in the Clegett Wines vineyard in Langhorne Creek in South Australia and it was registered under the name Malian. In 1991, one of those vines produced white berries, now registered as Shalistin. Clegett make a sweet and a dry version of Shalistin.
Cabernet Sauvignon was used to breed numerous varieties around the world, some of them cultivated for commercial winemaking. The following are therefore all half-siblings:
– Australia: CIENNA and TYRIAN
– Bulgaria: RUEN
– France: ARINARNOA, EKIGAÏNA and MARSELAN
– Germany: CABERNET CARBON, CABERNET CAROL, CABERNET CORTIS,
CABERNET CUBIN, CABERNET DORIO, CABERNET DORSA, CABERNET
MITOS
– Italy: INCROCIO MANZONI 2.15
– Serbia: PROBUS
– Romania: Arcas
– Switzerland: ?CABERNET BLANC, CABERNET COLONJES, CABERNET JURA,
?CABERTIN, CARMINOIR, ?PINOTIN, RIESEL and VB 91-26-4
– Ukraine: ODESSKY CHERNY and RUBINOVY MAGARACHA
– US: CARMINE and RUBY CABERNET.
Since the word sauvignon derives from the French sauvage (‘wild’), it has been argued that Cabernet Sauvignon was domesticated from wild grapevines in the Gironde, notwithstanding the fact that Vitis vinifera subsp. silvestris has never been recorded in this area.
Like CABERNET FRANC, Cabernet Sauvignon, one of whose early synonyms was Bidure, has been said to derive from the Biturica of Roman times but this theory can be rejected now that its parentage is known. Assuming that Biturica was identical or similar to Balisca, another variety mentioned by Pliny the Elder and Columella and supposedly cultivated in Spain under the name Cocolubis, some authors suggested that Cabernet Sauvignon was introduced from Spain.
A recent essay created quite a buzz by claiming that Cabernet Sauvignon was of Greek origin, its ancestor being the obscure Greek grape variety Volitsa (VOLITSA MAVRI; Lambert-Gocs 2007). Based on the vague descriptions given by Pliny the Elder and Columella, the author concluded that the modern Volitsa of the Pelopónnisos (Peloponnese), supposedly identical to a variety called VLOSH in Albania, is also identical to the antique Balisca and is therefore a direct ancestor of Cabernet Sauvignon, hence the Greek-origin hypothesis for the noble Bordeaux variety. However, DNA profiling is categorical: there is no direct parentage, nor any close genetic similarity, between Volitsa and Cabernet Sauvignon (Vouillamoz).
VITICULTURAL CHARACTERISTICS
Vigorous, late budding, mid to late ripening. Thrives on well-drained gravel soils, preferably acid and well exposed. Very susceptible to fungal diseases that affect the wood such as eutypa dieback and esca and also to powdery mildew. Occasionally susceptible to drying out of the stems (especially when grafted onto SO4 rootstock). Bunches and berries are small, the skins thick and distinctively blueish. Its hard wood makes it suitable for mechanical harvesting and generally safe from the risk of winter freeze.
WHERE IT’S GROWN AND WHAT ITS WINE TASTES LIKE
Because Cabernet Sauvignon is the dominant grape in the assemblages responsible for all of the most famous wines made (in quantity) in Bordeaux’s Médoc and Graves regions, including all five ‘first growths’, Cabernet Sauvignon has been admired and planted almost wherever vines are grown for wine production – except in regions too cool for this relatively late ripener. This global spread was particularly marked during the late-twentieth-century’s love affair with international varieties.
It has a particularly powerful identity. Thanks to the small, thick-skinned grapes, the wines tend to be deep in colour, high in tannin and relatively high in acidity. There’s a recognizable nobility and stability to its aroma which can vary from blackcurrant to cedar, although such is its affinity with oak that in young wines it can be difficult to dissociate the fruit from the wood. Few varieties produce wines with a higher level of phenolics and winemakers have to be particularly careful with their extraction techniques in some vintages – especially when Cabernet Sauvignon grapes have had to struggle to ripen. The upshot of all this tannin is that Cabernet-based wines can last for decades, occasionally centuries, and may need, like that other tannic wine, vintage port, to be given many years’ bottle age before being palatable, or at least pleasurable. This is particularly true in climates as moderate as Bordeaux’s. Cabernet Sauvignon can be a mite monolithic, and often austere, in youth.
Because, unless very ripe, Cabernet Sauvignon tends to produce wines long on structure and rather shorter on flesh, it has traditionally been blended with the much fruitier, softer MERLOT. And because growing seasons can vary markedly in Bordeaux, the late Cabernet Sauvignon was traditionally planted with the more precocious Merlot and CABERNET FRANC as an insurance policy against poor fruit set. The close genetic relationship between these three varieties makes them easy blending partners and in wine parlance ‘Bordeaux blend’ tends to mean Cabernet Sauvignon with
 any of Merlot, Cabernet Franc and/or the other varieties grown to produce red bordeaux: PETIT VERDOT, Malbec (COT) and occasionally CARMENÈRE.
In warmer climes, especially in northern California and Washington State, Cabernet Sauvignon grapes can reach such an advanced state of physiological ripeness, and have been encouraged to do so by extended ‘hang time’ on the vine, that in a typical vintage they hardly need the added flesh of a MERLOT component in the blend, although some winemakers add a seasoning of other grape varieties for aromatic complexity.
What is remarkable about Cabernet Sauvignon is that it imprints its identity so firmly on any wine that includes it, even in quite small amounts, although it manages to produce great wine in only a small proportion of the hundreds of regions in which it is planted. The most obvious established candidates are the Médoc, Pessac-Léognan, Penedès, Napa, Bolgheri, Sonoma, Santa Cruz Mountains, parts of Washington State, Puente Alto in Chile, and Coonawarra and Margaret River in Australia.
In France, Cabernet Sauvignon was the fourth most planted red wine variety after MERLOT, Grenache (GARNACHA) and SYRAH in 2009, with a total of 56,386 ha (139,333 acres). In the greater Bordeaux region, it was a poor second to the much easier to ripen Merlot: 26,790 ha (66,200 acres) to Merlot’s 69,053 (170,634 acres).
Throughout south-west France, it is one of the principal varieties in appellations on the fringes of Bordeaux such as Buzet, Bergerac and Côtes de Duras and also in Irouléguy on the western Spanish border. It is a supplementary variety in the likes of Béarn, Madiran, Côtes de Saint-Mont, Tursan, Marcillac, Gaillac and Fronton (adding structure to NÉGRETTE in the last two).
It is the fifth most planted variety in Languedoc-Roussillon, with a total of 18,722 ha (46,263 acres) in 2009, mainly in the Languedoc. While it’s a permitted ingredient in appellations such as Cabardès and Malepère, it is used mainly for varietal wines. Perhaps the most famous is Mas de Daumas Gassac, where Cabernet Sauvignon was planted on the advice of the late Bordeaux oenologist Émile Peynaud, no less. With Aniane neighbour Grange des Pères, these are two of the most successful Cabernet Sauvignon-based wines from the Languedoc, but too often less ambitious examples lack conviction and concentration, perhaps because of excessive yields or poor-quality clones.
Cabernet Sauvignon grown to the east in Provence – 3,483 ha (8,607 acres), mainly in the southern Rhône and the Var – seems much more convincing and generally has no shortage of concentration. Indeed it sometimes has a sort of dry-climate dustiness. It has traditionally been blended with SYRAH in reds and rosés, although there is more Syrah planted here and much more Grenache (GARNACHA). Domaine de Trévallon is the standard-bearer.
In the Loire and central France, Cabernet Sauvignon’s total of 1,283 ha (3,170 acres) makes it very minor indeed compared with CABERNET FRANC (less than a tenth as much), partly because it is much harder to ripen this far from the equator, but Cabernet Sauvignon is the principal variety in Cabernet d’Anjou rosé, can be significant in appellations such as Anjou-Villages and is permitted (usually playing a bit part to Cabernet Franc’s lead) in Bourgueil and Chinon.
Cabernet Sauvignon is important in Italy, where it has been grown since the 1820s, when it was introduced to Piemonte. As in Spain, Italy has seen a period of Cabernet worship, particularly in the revitalized Maremma, where the variety really does seem at home, as evinced by Sassicaia, based on cuttings from Bordeaux, as long ago as the late 1960s. The variety was enthusiastically planted all over Italy in the late twentieth and early twenty-first centuries – particularly in Toscana, where it, along with MERLOT and occasionally SYRAH, was thought to add body and international appeal to what were seen as the shortcomings of the local SANGIOVESE. This was perfectly legal in Chianti Classico, not so for Brunello di Montalcino, which in 2008 was riven by ideological dispute over this issue. Cabernet Sauvignon, long a required ingredient in the Tuscan Carmignano DOC, was a vital element in many of the so-called Supertuscans that followed in Sassicaia’s footsteps and enjoyed fame in the late twentieth century, such as Solaia, Sammarco, Saffredi, Paleo Rosso, La Vigna d’Alceo and hundreds more. To a much more limited extent it was also planted throughout northern Italy, where its

 noblest examples include Gaja’s Darmagi in Piemonte, Maurizio Zanella in Lombardia, Lageder and San Leonardo in Alto Adige. Here and in Friuli it can be quite light and herbaceous.
The most recent official Italian viticultural census, in 2000, found 8,042 ha (19,872 acres) but the current total is likely to be a good deal higher, especially considering the fact that Sicilia alone had 3,992 ha (9,864 acres) planted in 2008. There are many DOCs with Cabernet in their name throughout northern and central Italy as well as further south in Puglia and Sicilia, some specifying Cabernet Sauvignon or CABERNET FRANC, some leaving it to the discretion of the producer. Cabernet Sauvignon-dominated wines of great quality are relatively rare this far south but Sicilia can boast examples from Planeta and Tasca d’Almerita, and the Montevetrano blend from Campania has also attracted international attention.
Cabernet Sauvignon was also introduced into Spain to a limited extent in the mid nineteenth century by the Marqués de Riscal in Rioja and Eloy Lecanda of Vega Santa Cecilia (later Vega Sicilia) in Ribera del Duero. But it was its espousal by Miguel Torres, Jr, and Jean León in the 1960s in Penedès that encouraged Spanish wine producers to value this French intruder so highly. A Torres, Mas La Plana Cabernet Sauvignon 1970, did remarkably well in a 1979 rerun of the famous 1976 Judgement of Paris blind-tasting comparison between top French wines and those emulating them elsewhere.
In 2008 Cabernet Sauvignon was the fourth most planted red wine variety in Spain, its total of 19,430 ha (48,013 acres) being a considerable increase on the 10,000 ha (24,710 acres) in the ground in 2004. It is particularly popular in such northern DOs as Navarra and Somontano and in La Mancha but it has become a significant minor ingredient with TEMPRANILLO in Ribera del Duero, and varietal Cabernet Sauvignon can be found all over the country, including in the newer high-altitude vineyards of Andalucía. Many of those who have planted new vineyards in Spain seem to view Cabernet Sauvignon as an essential accoutrement of sophistication, although only in Penedès does it have much of a track record. Also in Catalunya, it can be found in some Priorat blends too.
The Portuguese are more inured to Cabernet-mania and there were just 2,271 ha (5,612 acres) planted in 2010 in Portugal. Palmela on the Península de Setúbal is the only denomination that embraces it but it pops up in various Vinho Regionals all over the country, most successfully in the Tejo.
In Eastern Europe, Cabernet Sauvignon is very important in Bulgaria, where with a 2009 total of 15,827 ha (39,109 acres) it accounts for 15% of the country’s vineyard area and is almost as widely planted as the common PAMID. It is planted mainly in the south, although the wines tend to show more finesse in the north. Single-estate Bordeaux blends are the most popular wines produced by the new wave of ambitious Bulgarian producers.
Cabernet Sauvignon is widely planted in Ukraine – 8,468 ha (20,925 acres) in 2009 – and it is also a hugely popular variety over the Black Sea to the east in Russia, where there were 3,525 ha (8,710 acres) in Krasnodar Krai alone in 2010, as well as a more modest 68 ha (168 acres) in Rostov, and over the border to the west in the Republic of Moldova, where 7,590 ha (18,755 acres) were planted. Romania had 3,128 ha (7,729 acres) planted in 2008 but MERLOT is much more popular. All of these countries are capable of making very respectable wines from the variety but have until recently been hampered by poor viticulture and sometimes excessive yields.
Cabernet grown in Slovenia is generally a much more serious proposition, as demonstrated by the likes of Rojac in Istra (Istria) and Scurek and Marjan Simčič in Brda. In 2009 there were 470 ha (1,161 acres) planted, mainly in Vipavska Dolina and Goriška Brda near the Italian border but also further south near the coast in Slovenska Istra.
In Croatia, there were 894 ha (2,209 acres) in 2009, less than 3% of the country’s vines, while Cabernet’s 228 ha (563 acres) in the much, much cooler Czech Republic are predominantly in Morava in the south east, where it is most likely to stand a chance of ripening. There is a tiny amount planted in Slovakia – and indeed an increasing amount to the north in Germany, where global warming has allowed some growers such as Hammel and Martin Wassmer in the southern regions of

 the Pfalz and Baden respectively to ripen it successfully.
The warmer Pannonian climate of Hungary is much more suitable for this late-ripener even
though CABERNET FRANC is the favoured variety here. In 2008 Hungary had a total of 2,752 ha (6,800 acres), with the largest areas in Villány (where star performers include Malatinszky and Atilla Gere) and Eger, where it is a permitted ingredient in Egri Bikavér (Bull’s Blood of Eger).
Across the border in Austria, Cabernet Sauvignon’s 519 ha (1,282 acres) made it the third most planted red wine grape in 2007 but it is far less important than BLAUFRÄNKISCH and ZWEIGELT and is generally used as an ingredient in a blend, or cuvée. Plantings are mainly in Burgenland, particularly in Neusiedlersee (151 ha/373 acres), where the area has almost doubled since 1999. In Switzerland, too, fully ripe Bordeaux blends, and blends with indigenous Swiss varieties, have become more prevalent thanks to global warming, not just in Ticino but in the French-speaking Swiss cantons, eg in Genève, where Domaine des Balisiers make their Comte de Penay cuvée in amphorae.
Cabernet Sauvignon has played a part in the modern wine revolution in Greece, where in 2008 there were 1,743 ha (4,307 acres), particularly in Dráma and Kavala in north-eastern Makedonía, Messinía on the west of the Pelopónnisos and Attikí south of Athens. Antonopoulos, Biblia Chora, Costa Lazaridi, Pavlidis, Katogi Strofilia, Tsantali and Tselepos have all made very creditable examples. It plays a more minor role on the island of Cyprus, where there are only 279 ha (689 acres). The tiny Mediterranean island of Malta boasts about 100 ha (247 acres).
As Turkey’s wine industry has joined the mainstream, Cabernet Sauvignon has increasingly been planted, so that by 2010 there were 493 ha (1,218 acres) and some pretty serious blends, such as those of Corvus, Büyülübağ and Kayra.
Cabernet Sauvignon has played a crucial role in the modern wine industry in Lebanon, not least for providing the backbone of its best-known export, Château Musar’s spicy red. The variety is also, not surprisingly, the most popular variety in Israel’s cosseted vineyards, where, with a 2009 total of 1,250 ha (3,089 acres), it constitutes a quarter of all plantings and is particularly successful in the hands of Ella Valley, Galil Mountain, Golan Heights and Margalit.
Cabernet’s popularity in Israel is a reflection of its huge importance in the United States. With the exception of a short-term fling with MERLOT and a Sideways-inspired flirtation with PINOT NOIR, Cabernet Sauvignon is red wine for Americans. It is planted in every state that can ripen it and in California, the nation’s dominant wine producer, is the most planted red wine variety by far, its 2010 total of 77,602 acres (31,404 ha) second only to CHARDONNAY. It grows like a sprawling weed in the Central Valley and is planted the length of the state but reaches its apogee in Napa Valley’s sun- kissed, fog-cooled climes as well as in Alexander Valley, Sonoma Valley and various other spots protected from exaggerated Pacific influence, such as on well-exposed sites above the fog line of the Santa Cruz Mountains. If price is any measure, then California has definitively proved to be one of Cabernet Sauvignon’s sweetest spots. The dotcom boom fuelled demand for limited-production, so- called cult Cabernets, so that for a time they sold like hot cakes via over-subscribed mailing lists. So ripe are many of these wines that they can be drunk on release and some taste so sweet and alcoholic that they can sometimes seem, to a palate raised on the more angular charms of red bordeaux, closer to hot chocolate than wine.
The variety is also grown in Mexico, particularly in Baja California, Coahulia de Zaragoza and Zacatecas.
Much of Oregon is too cool to ripen Cabernet of either sort, except for the Columbia River region in the north east, on the border with Washington State and, to a lesser extent, in the Rogue Valley in the south west, near the California border. In 2008 a total of 523 acres (212 ha) was planted. In semi- desert eastern Washington State, on the other hand, Cabernet Sauvignon has become by far the most popular variety, with plantings totalling 10,293 acres (4,167 ha) by 2011. Here Cabernet Sauvignon can really shine, the fruit often showing appetizing brightness. Leading vineyards include Champoux in Horse Heaven Hills, Ciel du Cheval and Klipsun vineyards in Red Mountain, and Wahluke Slope. Whereas in California there is an army of admired producers of Cabernet Sauvignon, Washington’s

 constitute more of a cohort, with Leonetti, Powers, Quilceda Creek, Andrew Will and Woodward Canyon in the front line but dozens more only a step behind.
Virtually all American states make Cabernet Sauvignon and the majority of them grow it, with New York State’s 340 acres (138 ha) mainly on Long Island, and the Texas wine industry is dominated by it. Stylistically most states’ winemakers are heavily influenced by the California style of ripe fruit given a bit of discipline in expensive French, and occasionally American, oak. Virginia’s 260 acres (105 ha) of Cabernet Sauvignon are a little more difficult to ripen than the CABERNET FRANC favoured by many growers, although RdV have proved that, with care, Virginia Cabernet Sauvignon can be magnificent.
In Canada, the balmy temperatures of the southern Okanagan Valley in British Columbia are much better suited to Cabernet Sauvignon than anywhere else and the BC total in 2008 was 1,683 acres (681 ha), representing nearly 15% of red wine varieties. Mission Hill’s top bottling and Osoyoos Larose are outstanding Cabernet-based wines by any measure. Ontario had only about a third as much in 2006 and very careful site selection is needed here for this late-ripening variety.
Cabernet Sauvignon is also hugely important to South American wine – especially in Chile, where cuttings were brought directly from Bordeaux before phylloxera. The 2008 total of 40,728 ha (100,641 acres and far more even than in California) represents about a third of all vines planted and the variety can be found in all regions, especially in Colchagua, Maule, and to a lesser extent in Curicó, Maipo and Cachapoal, where it is the most planted variety. It was notable that some of the world’s most garlanded producers of Cabernet Sauvignon-dominated wines elsewhere – the likes of Robert Mondavi of California and Baroness Philippine de Rothschild, Bruno Prats and Paul Pontallier of Bordeaux – chose to invest in Chilean Cabernet Sauvignon many years ago. Chilean Cabernet, much of it grown on ungrafted vines in a disease-free environment, tastes particularly fruity and easy to appreciate young, even if there can be an edge of greenness to wines from over-burdened vines. But there is also a deepening top layer of more complex, ageworthy examples such as Almaviva, Altaïr, Aurea Domus, Casa Lapostolle, Concha y Toro’s Don Melchior, De Martino, Errazuriz/Viñedo Chadwick/Seña, and top bottlings from the likes of Gilmore, Haras de Pirque, Montes, O Fournier, Perez Cruz and others.
In Argentina, Malbec (COT) and Bonarda (DOUCE NOIRE) are much more important than Cabernet Sauvignon, which is a shame since Argentine Cabernet can present a magnificent combination of ripeness and minerality (and occasionally tart greenness if yields are too high). In 2008 there was a total of 17,746 ha (43,851 acres), the majority (13,342 ha/32,968 acres) in Mendoza, but it is also planted in several other regions, notably San Juan (2,182 ha/5,392 acres), La Rioja (1,018 ha/2,515 acres) and Salta (444 ha/1,097 acres). It is also often used as a stiffener for Malbec. Some of those who produce superior Cabernet Sauvignons include Catena Zapata, Viña Cobos, Cuvelier Los Andes, Mendel, Terrazas de los Andes, Fabre Montmayou and Vistalba, most of them having close links to either Bordeaux or the Napa Valley.
Brazil had 1,868 ha (4,616 acres) of Cabernet Sauvignon in 2007 but it can be difficult to ripen it fully in traditional damp regions, although Lidio Carraro, Miolo and Salton have all made creditable examples.
TANNAT and even MERLOT are more important in Uruguay, which had 731 ha (1,806 acres) of Cabernet Sauvignon in 2008.
In the 1970s and 1980s, Cabernet Sauvignon was worshipped in Australia but it is now viewed as a poor second to Shiraz (SYRAH), both qualitatively and quantitatively. In 2008 the total area planted with Cabernet Sauvignon was 27,553 ha (68,085 acres), considerably less than Shiraz (SYRAH). By far the majority of plantings are in the Riverland (3,496 ha/8,639 acres), for inexpensive, fruity varietals, and in Coonawarra (3,444 ha/8,510 acres), which was the first region in Australia ever to establish a varietal speciality, Cabernet Sauvignon. In recent years a lack of fine-tuned viticulture has tarnished Coonawarra Cabernet’s reputation but the potential for top-quality wine from its terra rossa soils is indubitably there. Cabernet Sauvignon is also important in Langhorne Creek (1,921 ha/4,747

acres), which produces a rather soft style for blending, Barossa Valley (1,336 ha/3,301 acres), whose Cabernet is particularly dense and chocolatey, and Western Australia’s Margaret River (1,291 ha/3,190 acres), which makes sophisticated, well-balanced examples – on a west coast just like the variety’s other homes in Bordeaux, Napa Valley and Bolgheri. (There is a theory that rays from the setting sun are particularly propitious for Cabernet Sauvignon vines.) The Margaret River tradition is to blend Cabernet Sauvignon with MERLOT for less serious bottlings and to make varietal Cabernet Sauvignon for the long term. Australia’s most accomplished Cabernet Sauvignon producers include Cape Mentelle, Cullen, Howard Park, Moss Wood, Vasse Felix and Woodlands in Margaret River; Balnaves, Bowen, Majella, Parker and Wynns in Coonawarra; Henschke, Penfolds and Wendouree from further north in South Australia; Mount Mary and Yarra Yering in Victoria; Lake’s Folly in Hunter Valley. Cabernet Sauvignon/Shiraz blends are part of Australian wine’s long and glorious history.
New Zealand is PINOT NOIR not Cabernet country. There were only 517 ha (1,278 acres) of Cabernet Sauvignon vines in the ground in 2009 – much less than Pinot Noir’s 4,240 ha (10,477 acres) and way behind MERLOT’s 1,363 ha (3,368 acres). Most regions are too cool or wet for the variety but it clearly thrives in the appropriately named Gimblett Gravel soils and elsewhere in Hawke’s Bay on the North Island. Reliable producers of varietal Cabernet Sauvignon and, generally more successful, Bordeaux blends include Cape Kidnappers, Craggy Range, Esk Valley, C J Pask, Stonecroft, Stonyridge, Te Awa Farm, Te Mata, Trinity Hill, Unison and Villa Maria.
As for China, the world’s fastest-growing wine producer, Cabernet Sauvignon has played the crucial role, being by far the most favoured variety since red bordeaux has been seen as the archetypal wine. By 2009 plantings were reckoned to total 20,352 ha (50,288 acres), in all of China’s wine regions, and they must already be considerably more than this – very possibly more than in Bordeaux by now. Inland Ningxia is looking like the province with the greatest potential, although the Rothschilds of Château Lafite have bet on Shangdong on the coast. Much of the wine produced is extremely lean but producers such as Grace, Silver Heights and Jade Mountain have broken the mould.
The variety has been planted in virtually all new territories for the vine, including Japan (469 ha/1,159 acres in 2009, mostly in Yamanashi and Hyogo), which, with extreme vine-by-vine care, can produce quite creditable copies of ripe red bordeaux from its limited plantings. In 1988 Grover pioneered Cabernet Sauvignon in India, with several harvests a year on high ground north of Bangalore in Karnataka State.
Cabernet Sauvignon is taken very seriously in South Africa, even if work has been needed to eradicate the ripening difficulties associated with the leafroll virus endemic in some vineyards. Cabernet Sauvignon is traditionally blended with MERLOT here and in 2008 total plantings were 12,697 ha (31,375 acres), making Cabernet Sauvignon the Cape’s most widely planted red wine variety, accounting for 28% of red plantings and second only to CHENIN BLANC overall. The main areas are Stellenbosch, Paarl and Malmesbury, and those making particularly distinctive Cabernets include Glenelly Hill, Jordan, Morgenster, Reyneke, Rustenberg, Simonsig, Thelema, Vergelegen, Vilafonté, Warwick and Waterford.
'''


In [13]:
keywords = kw_extract(text)
print(keywords)

cabernet, sauvignon, wine, acre, variety, ha, plant, bordeaux, much, make
